In [1]:
from datetime import datetime, timedelta
from quantlib.datasource.service import DataService

In [2]:
# Create service with Alpha Vantage
service = DataService.create_default_service()

# Get data
end_date = datetime.now()
start_date = end_date - timedelta(days=30)

C:\Users\enzob\OneDrive\Desktop\ai\quantlib\quantlib\datasource\sources\mt5.py:40: UserWarning: Failed to initialize MT5: (-6, 'Terminal: Authorization failed')
  warnings.warn(f"Failed to initialize MT5: {mt5.last_error()}")


In [3]:
service.list_sources()

['yahoo_finance', 'mt5']

In [4]:
intraday_data = service.get_close_prices(
    'PETR4',
    'mt5',
    '2025-01-01',
    '2025-01-10',
    interval='1h',
)
intraday_data.data

RuntimeError: MT5 is not initialized

In [1]:
import pandas as pd
from functools import reduce
path = r"C:\Users\enzob\AppData\Roaming\MetaQuotes\Terminal\D0E8209F77C8CF37AD8BF550E51FF075\MQL5\Profiles\Tester\algo\price\WIN_PROFIT_FACTORY\\"
strategies = ['maxproffitfactor.csv', 'proffitfactorbalance.csv', 'proffitfactorlowstd.csv', 'profitfactormeanreversion.csv']

strats_dfs = {}
for strategy in strategies:
    strats_dfs[strategy] = pd.read_csv(path + strategy, sep='\t', decimal=',', encoding='UTF-16LE')
    strats_dfs[strategy][strategy.replace('.csv', '')] =  strats_dfs[strategy]['<BALANCE>'].astype(float)
    strats_dfs[strategy]['<DATE>'] = pd.to_datetime(strats_dfs[strategy]['<DATE>'], format='%Y.%m.%d %H:%M')
    strats_dfs[strategy] = strats_dfs[strategy][['<DATE>', strategy.replace('.csv', '')]]

merged_df = reduce(lambda left, right: pd.merge(left, right, on='<DATE>', how='outer'), strats_dfs.values())
merged_df.fillna(method='ffill', inplace=True)
merged_df['total'] = merged_df[merged_df.columns[1:]].sum(axis=1)
merged_df['<DATE>'] = pd.to_datetime(merged_df['<DATE>'].dt.date)
merged_df = merged_df.groupby('<DATE>').last()


C:\Users\enzob\AppData\Local\Temp\ipykernel_16132\1576016680.py:14: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_df.fillna(method='ffill', inplace=True)


In [2]:
from quantlib.data.timeseries import TimeSeries
from quantlib.data.multitimeseries import MultiTimeSeries
import numpy as np

In [3]:
merged_df['total']

<DATE>
2020-05-08    401115.0
2020-05-11    401130.0
2020-05-12    400013.0
2020-05-13    396656.0
2020-05-14    394879.0
                ...   
2025-04-29    989680.0
2025-04-30    990321.0
2025-05-02    991403.0
2025-05-05    993295.0
2025-05-06    994400.0
Name: total, Length: 1238, dtype: float64

In [26]:
ts = MultiTimeSeries(merged_df)

In [27]:
ts.sharpe_ratio()

total    4.816848
dtype: float64

In [28]:
ts.max_drawdown()


total   -0.042507
dtype: float64

In [29]:
ts.max_drawdown(percentage=False)


total   -17050.0
dtype: float64

In [30]:
ts.value_at_risk()

total   -0.003829
Name: 0.05, dtype: float64

In [31]:
ts.value_at_risk(percentage=False)

total   -2697.2
Name: 0.05, dtype: float64

In [32]:
ts.autocorrelation()

total   -0.008656
dtype: float64

In [33]:
ts.kurtosis()


total    4.307276
dtype: float64

In [34]:
ts.skewness()

total   -0.505434
dtype: float64

In [35]:
ts.correlation()

AttributeError: 'TimeSeries' object has no attribute 'correlation'

In [36]:
ts.covariance()

AttributeError: 'TimeSeries' object has no attribute 'covariance'

In [37]:
ts.data

,total
<DATE>,
2020-05-08,401115.0
2020-05-11,401130.0
2020-05-12,400013.0
2020-05-13,396656.0
2020-05-14,394879.0
...,...
2025-04-29,989680.0
2025-04-30,990321.0
2025-05-02,991403.0


In [38]:
ts.rolling(window=10)

ValueError: If using all scalar values, you must pass an index

In [45]:
rolling_data = pd.DataFrame()
rolling_data['mean'] = ts.data.rolling(10, min_periods=10).mean()
rolling_data['std'] = ts.data.rolling(10, min_periods=10).std()
rolling_data['min'] = ts.data.rolling(10, min_periods=10).min()
rolling_data['max'] = ts.data.rolling(10, min_periods=10).max()

In [46]:
rolling_data

,mean,std,min,max
<DATE>,,,,
2020-05-08,NaN,NaN,NaN,NaN
2020-05-11,NaN,NaN,NaN,NaN
2020-05-12,NaN,NaN,NaN,NaN
2020-05-13,NaN,NaN,NaN,NaN
2020-05-14,NaN,NaN,NaN,NaN
...,...,...,...,...
2025-04-29,994992.0,2856.577323,989680.0,998883.0
2025-04-30,994325.7,3106.461476,989680.0,998883.0
2025-05-02,993738.8,3041.708248,989680.0,998883.0


: 

In [43]:
ts.rolling(window=10)


ValueError: If using all scalar values, you must pass an index

In [44]:
ts.data.rolling(10, min_periods=10).mean()

,total
<DATE>,
2020-05-08,NaN
2020-05-11,NaN
2020-05-12,NaN
2020-05-13,NaN
2020-05-14,NaN
...,...
2025-04-29,994992.0
2025-04-30,994325.7
2025-05-02,993738.8
